In [1]:
!pip install datasets
!pip install --upgrade jupyter ipywidgets



Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch, transformers
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset
import pandas as pd, numpy as np
from torch import cuda

In [3]:
!nvidia-smi

Fri May  3 13:04:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    On  | 00000000:00:1E.0 Off |                    0 |
|  0%   22C    P8               8W / 300W |      4MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
#global params for training

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", padding = True, return_tensors = "pt",is_split_into_words=True)
batch_size = 64
epoch = 100
max_tokenizer_len = 512


device = 'cuda' if cuda.is_available() else 'cpu'
print(device)




cuda


In [5]:
from datasets import load_dataset
import pandas as pd, numpy as np
    
dataset = load_dataset('knowledgator/events_classification_biotech') 


/opt/conda/lib/python3.8/site-packages/datasets/load.py:1486: FutureWarning: The repository for knowledgator/events_classification_biotech contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/knowledgator/events_classification_biotech
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'content', 'target organization', 'all_labels', 'all_labels_concat', 'label 1', 'label 2', 'label 3', 'label 4', 'label 5'],
        num_rows: 2759
    })
    test: Dataset({
        features: ['title', 'content', 'target organization', 'all_labels', 'all_labels_concat', 'label 1', 'label 2', 'label 3', 'label 4', 'label 5'],
        num_rows: 381
    })
})

In [7]:
train_df_temp = pd.DataFrame(dataset['train'])
test_df_temp = pd.DataFrame(dataset['test'])

In [8]:
cols = ['content' , 'all_labels']
train_df = train_df_temp[cols].copy()
test_df = test_df_temp[cols].copy()
    
    
    


In [9]:
distinct_values_count = train_df['all_labels']
#print(distinct_values_count)
all_vals = (list(distinct_values_count))
concatenated_list = [item for sublist in all_vals for item in sublist]
#print(concatenated_list)
list_labels = list(set(sorted(concatenated_list)))
print(list_labels)
list_labels.index('other')


['hiring', 'foundation', 'clinical trial sponsorship', 'alliance & partnership', 'article publication', 'patent publication', 'department establishment', 'subsidiary establishment', 'closing', 'new initiatives or programs', 'other', 'product launching & presentation', 'company description', 'executive statement', 'new initiatives & programs', 'funding round', 'partnerships & alliances', 'm&a', 'expanding geography', 'expanding industry', 'event organization', 'executive appointment', 'investment in public company', 'participation in an event', 'regulatory approval', 'product updates', 'support & philanthropy', 'service & product providing', 'ipo exit']


10

In [10]:
concatenated_list

['other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',


In [11]:
def create_label(all_labels):
    #print(all_labels)
    lab_list = [list_labels.index(item) for item in all_labels]
    return lab_list
    

In [12]:
train_df['labels'] = train_df['all_labels'].apply(create_label)

test_df['labels'] = test_df['all_labels'].apply(create_label)

In [13]:
train_df.columns

Index(['content', 'all_labels', 'labels'], dtype='object')

In [14]:
cols = ['labels','content']
na_rows = train_df[cols].isna().any(axis=1)


In [15]:
cleaned_df_train = train_df[~na_rows]

In [16]:
cleaned_df_train

,content,all_labels,labels
0,Drive Your Plow Over the Bones of The Dead\nby...,[other],[10]
1,"In the recently tabled National Budget, Denel ...",[other],[10]
2,Shares\nTake a break its good for you (Picture...,[other],[10]
3,RESO is currently hiring for two positions:\nP...,[other],[10]
4,Charter Buyer Club\nWhat is the Charter Buyer ...,[other],[10]
...,...,...,...
2754,0\nA regional daily wants to bring an internat...,"[support & philanthropy, company description]","[26, 12]"
2755,Share on whatsapp\nR1 RCM Inc. (NASDAQ:RCM)\nw...,"[investment in public company, company descrip...","[22, 12]"
2756,Verboso Launches Full-Stack Online Speech Ther...,[product launching & presentation],[11]
2757,"Barnet, Enfield and Haringey Mental Health Tru...",[executive statement],[13]


In [17]:
na_rows_test = test_df[cols].isna().any(axis=1)
cleaned_df_test = test_df[~na_rows_test]


# Train Test split
    
    

In [18]:
seed = 42
# Randomly shuffle the DataFrame
df_shuffled = cleaned_df_train.sample(frac=1, random_state=seed)
train_ratio = 0.8
# Calculate the number of samples for training
train_size = int(train_ratio * len(df_shuffled))

# Split the shuffled DataFrame into training and testing sets
df_train = df_shuffled[:train_size]
df_val = df_shuffled[train_size:]


In [19]:
# Test the tokenizer:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", padding = True, return_tensors = "pt" , truncate = True, max_length  = 512)

tokens = tokenizer(df_train['content'][100])
tokens


Token indices sequence length is longer than the specified maximum sequence length for this model (566 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': [101, 17781, 3225, 1130, 17030, 5430, 2825, 1605, 14866, 17087, 2553, 1111, 4108, 2328, 17564, 1348, 3225, 4326, 1203, 2470, 1116, 108, 122, 23089, 1419, 11458, 22233, 26616, 15468, 1106, 11125, 23089, 2616, 118, 6303, 1428, 1626, 117, 17881, 1477, 1367, 131, 3862, 27269, 197, 5313, 131, 17781, 3225, 2796, 20537, 117, 4565, 117, 7414, 12150, 10069, 23676, 13821, 9919, 18589, 2064, 2591, 4880, 24846, 2069, 4880, 24846, 117, 151, 119, 150, 119, 117, 13650, 119, 1626, 117, 17881, 1477, 113, 144, 2162, 2346, 27211, 26546, 18019, 25190, 16941, 114, 118, 118, 17781, 3225, 117, 1203, 2470, 1116, 108, 122, 2825, 1605, 14866, 1881, 117, 3055, 1850, 170, 2998, 1106, 1203, 2470, 1116, 3289, 2332, 22233, 24990, 1105, 1203, 2470, 1352, 7844, 1106, 5622, 15468, 1121, 22233, 24990, 1111, 23089, 5811, 1112, 170, 18560, 2332, 1555, 119, 1109, 4909, 1110, 170, 2593, 1106, 170, 2793, 1644, 1115, 5802, 1155, 2616, 118, 6303, 1105, 1251, 1149, 118, 1104, 118, 4480, 4692, 1111, 18560, 2332, 18

### Data loaders and Dataset for batched training

In [20]:
class dataset(Dataset):
    def __init__(self, df,len_labels):
        self.df = df
        self.len = len_labels
        
    def __getitem__(self, idx):
        text = self.df.iloc[idx]['content']
        x = tokenizer.encode_plus(text,max_length= max_tokenizer_len, padding="max_length",truncation=True, return_tensors="pt" )
        label_vec = np.zeros(self.len)       
        label = torch.tensor(self.df.iloc[idx]['labels'])
        label_vec[label] = 1
        #take the id of the row and return tokeinzed tensors
        #print(x.shape)
        x_dict = {}
        x_dict['id'] = x['input_ids']
        x_dict['token_type'] = x['token_type_ids']
        x_dict['attention_mask'] = x['attention_mask']
        
        
        return x_dict, torch.tensor(label_vec)
        
    def __len__(self):
        #return the length of the dataframe
        return len(self.df)
    

In [21]:
train_dataset = dataset(df_train,len(list_labels))
val_dataset = dataset(df_val,len(list_labels))
test_dataset = dataset(test_df,len(list_labels))

train_loader = DataLoader(train_dataset,batch_size = batch_size, shuffle = True , num_workers = 0, pin_memory = True)
val_loader = DataLoader(val_dataset,batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test_dataset,batch_size = batch_size, shuffle = False)

    

In [22]:
#enc = transformers.BertModel.from_pretrained('bert-base-uncased')
# ids = torch.tensor(tokens['input_ids'])
# t_type = torch.unsqueeze(torch.tensor(tokens['token_type_ids']), dim = 0)
# att_m = torch.unsqueeze(torch.tensor(tokens['attention_mask']), dim = 0)
# enc(ids, t_type,att_m, return_dict = False)

In [23]:
class clf_model(torch.nn.Module):
    def __init__(self):
        super(clf_model, self).__init__()
        self.backbone = transformers.BertModel.from_pretrained('bert-base-uncased')
        #self.norm_layer = torch.nn.LayerNorm()
        self.linear_layer = torch.nn.Linear(768,len(list_labels))
        self.sigmoid = torch.nn.Sigmoid()
 
                  
    def forward(self,ids ,token_type,att_mask):
        #print(f"shape of ids->{ids.shape}")
           #ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False     
        _,x = self.backbone(ids , token_type_ids =token_type ,attention_mask= att_mask, return_dict=False)
        
        #print(f"shape after backbone ->{x.shape}")
        #print(f"x after backbone ->{x}")
                     
        x = self.linear_layer(x)
        #print(f"shape after linear_layer->{x.shape}")
        #print(f"x after linear ->{x}")
        layer_norm = torch.nn.LayerNorm(x.shape[1], device = 'cuda:0')
        x = layer_norm(x)
        #print(f"shape x after layer norm ->{x.shape}")
        #print(f"x after layer normm ->{x}")
        
        logits = self.sigmoid(x)
        del x
        
        #print(f"shape of logits->{logits.shape}")
        #print(f"logits = {logits}")
        return logits
    
        
        
        

In [24]:
# tokens 
# x_temp = torch.tensor(tokens['input_ids'][:512]).unsqueeze(dim = 0)
# print(x_temp.shape)
# x_att = torch.tensor(tokens['attention_mask'][:512]).unsqueeze(dim = 0)
# x_tok_type = torch.tensor(tokens['token_type_ids'][:512]).unsqueeze(dim = 0)

# x_temp_gpu = x_temp.to(device)
# x_att_gpu = x_att.to(device)
# x_tok_type_gpu = x_tok_type.to(device)



# m_gpu  = transformers.BertModel.from_pretrained('bert-base-uncased').to(device)
# m_gpu.to(device)
# m_cpu = transformers.BertModel.from_pretrained('bert-base-uncased')



In [25]:
# o_gpu = m_gpu(x_temp_gpu,x_att_gpu,x_tok_type_gpu)
# o_gpu

In [26]:
# m_clf = clf_model()
# m_clf.to(device)
# t = m_clf(x_temp_gpu,x_tok_type_gpu,x_att_gpu)


In [27]:
# train_iter = iter(train_loader)
# for batch in train_loader:
#     x_dict, y = batch
#     ids = torch.squeeze(torch.tensor(x_dict['id']).clone().detach(),dim = 1)
#     print(f"shape of ids->{ids.shape}")
#     token_type = torch.squeeze(torch.tensor(x_dict['token_type']).clone().detach(), dim = 1)
#     print(f"shape of token_type->{token_type.shape}")
#     att_mask = torch.squeeze(torch.tensor(x_dict['attention_mask']).clone().detach(), dim = 1)
#     print(f"shape of att_mask->{att_mask.shape}")
#     logits = model(ids ,token_type,att_mask)
#     print(logits)    
#     print(y.shape)
#     #input_id = batch['input_ids']
    
    
    

In [28]:
# train_iter = iter(train_loader)
# for batch in train_loader:
#     x_dict, y = batch
#     model.to(device)
#     #print(next(model_gpu.parameters()).device)
#     ids = torch.squeeze(torch.tensor(x_dict['id']).clone().detach(),dim = 1)
#     print(f"shape of ids->{ids.shape}")
#     token_type = torch.squeeze(torch.tensor(x_dict['token_type']).clone().detach(), dim = 1)
#     print(f"shape of token_type->{token_type.shape}")
#     att_mask = torch.squeeze(torch.tensor(x_dict['attention_mask']).clone().detach(), dim = 1)
#     print(f"shape of att_mask->{att_mask.shape}")
#     ids_gpu = ids.to(device)
#     print(f"ids = {ids_gpu}")
#             #ids.to(device)
#     print(ids_gpu.device)
#     token_type_gpu = token_type.to(device)
#     print(token_type_gpu.device)
#     att_mask_gpu = att_mask.to(device)
#     print(att_mask_gpu.device)
#     #label_vec.to(device)
#     logits = model(ids_gpu ,token_type_gpu,att_mask_gpu)
#     print(logits)
    

### Training the model

In [29]:
len(train_loader)

35

In [30]:
def train_model(train_loader,optimizer, num_epoch = 100, model = clf_model()):
    model.train()
    for i in range (num_epoch):
        loss_list = []
        for ind,(x_dict, label_vec) in enumerate(train_loader):
            model.to(device)
            #print(next(model_gpu.parameters()).device)
            
            id_list = x_dict['id']
            id_tensor = torch.squeeze(torch.tensor(id_list),dim = 1)
            #print(f"shape of ids->{id_tensor.shape}")
            
            tok_type_list = x_dict['token_type']
            tok_type_tensor = torch.squeeze(torch.tensor(tok_type_list),dim = 1)
            
            #token_type = torch.squeeze(torch.tensor(,dtype=torch.long).clone().detach(), dim = 1)
            #print(f"shape of token_type->{tok_type_tensor.shape}")
        
            att_list = x_dict['attention_mask']
            att_mask_tensor = torch.squeeze(torch.tensor(att_list),dim = 1)
            #att_mask = torch.squeeze(torch.tensor(,dtype=torch.long).clone().detach(), dim = 1)
            #print(f"shape of att_mask->{att_mask_tensor.shape}")
                        
            ids_gpu = id_tensor.to(device)
            #print(f"ids = {ids_gpu}")
            #ids.to(device)
            #print(ids_gpu.device)
            
            token_type_gpu = tok_type_tensor.to(device)
            #print(token_type_gpu.device)
            
            att_mask_gpu = att_mask_tensor.to(device)
            #print(att_mask_gpu.device)
            
            
            
            labal_vec_gpu = label_vec.float().to(device)
            #print(f" labal_vec_gpu = {labal_vec_gpu}")
            
            logits = model(ids_gpu ,token_type_gpu,att_mask_gpu)
            optimizer.zero_grad(set_to_none=True)
            loss = nn.BCELoss()
            output = loss(logits, labal_vec_gpu)
            
            #loss = torch.nn.BCEWithLogitsLoss(logits, labal_vec_gpu)
            output.backward()
            optimizer.step
            loss_list.append(output.item())
            #print(f"batch_ind{ind} and its loss is {loss.item()}")
            
        
        print(f" epoch= {num_epoch} and mean train loss is {torch.mean(torch.tensor(loss_list))}")
    
    return model
        
            
            
    
    

In [31]:
model = clf_model()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-3)
tr_model = train_model(train_loader, optimizer )

/tmp/ipykernel_605/591830481.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  id_tensor = torch.squeeze(torch.tensor(id_list),dim = 1)
/tmp/ipykernel_605/591830481.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tok_type_tensor = torch.squeeze(torch.tensor(tok_type_list),dim = 1)
/tmp/ipykernel_605/591830481.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  att_mask_tensor = torch.squeeze(torch.tensor(att_list),dim = 1)


 epoch= 100 and mean train loss is 0.8235846161842346
 epoch= 100 and mean train loss is 0.8234076499938965
 epoch= 100 and mean train loss is 0.8234884738922119
 epoch= 100 and mean train loss is 0.8231267333030701
 epoch= 100 and mean train loss is 0.8236042857170105
 epoch= 100 and mean train loss is 0.8236517906188965
 epoch= 100 and mean train loss is 0.8237167000770569
 epoch= 100 and mean train loss is 0.8237625956535339
 epoch= 100 and mean train loss is 0.8233580589294434
 epoch= 100 and mean train loss is 0.8233585357666016


In [32]:
torch.cuda.is_available()

True

In [33]:
import torch.nn as nn
m = nn.Sigmoid()
loss = nn.BCELoss()
input = torch.randn(3, 2, requires_grad=True)
print(f"input = {input}")
target = torch.rand(3, 2, requires_grad=False)
print(f"target = {target}")
print(f"sigmoid out = {m(input)}")
output = loss(m(input), target)
print(f"output={output} ")
output.backward()

input = tensor([[ 1.5612,  1.4753],
        [-1.3181, -2.5301],
        [-1.2903,  0.9343]], requires_grad=True)
target = tensor([[0.4247, 0.8626],
        [0.3439, 0.4488],
        [0.3545, 0.6084]])
sigmoid out = tensor([[0.8265, 0.8139],
        [0.2111, 0.0738],
        [0.2158, 0.7180]], grad_fn=<SigmoidBackward0>)
output=0.7995781898498535 


In [34]:

x_dict,_ = train_dataset[85]
#print(x)
y = torch.tensor([[0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

print(y)
print(f"y dtype ={y.dtype}")
m_gpu = clf_model()
m_gpu.to(device)
id_list = x_dict['id']
id_tensor = torch.squeeze(torch.tensor(id_list),dim = 1)
tok_type_list = x_dict['token_type']
tok_type_tensor = torch.squeeze(torch.tensor(tok_type_list),dim = 1)
att_list = x_dict['attention_mask']
att_mask_tensor = torch.squeeze(torch.tensor(att_list),dim = 1)
ids_gpu = id_tensor.to(device)
token_type_gpu = tok_type_tensor.to(device)
att_mask_gpu = att_mask_tensor.to(device)
labal_vec_gpu = y.to(device)

logits = m_gpu(ids_gpu ,token_type_gpu,att_mask_gpu)
print(logits.dtype)
loss = nn.BCELoss()
output = loss(logits, labal_vec_gpu)
output.backward()
print(output)


tensor([[0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
y dtype =torch.float32
torch.float32
tensor(0.7851, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)


/tmp/ipykernel_605/191361512.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  id_tensor = torch.squeeze(torch.tensor(id_list),dim = 1)
/tmp/ipykernel_605/191361512.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tok_type_tensor = torch.squeeze(torch.tensor(tok_type_list),dim = 1)
/tmp/ipykernel_605/191361512.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  att_mask_tensor = torch.squeeze(torch.tensor(att_list),dim = 1)


In [35]:
y_test = torch.tensor([[0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
print(y_test.shape)

torch.Size([1, 29])
